In [74]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [75]:
columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income" ]

dataset = pd.read_csv("datasets/datasets/adult/raw/adult.data", header=None, sep=",\s", names=columns, na_values=["?"], engine="python")

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_8732/2698192861.py:3: SyntaxWarning: invalid escape sequence '\s'
  dataset = pd.read_csv("datasets/datasets/adult/raw/adult.data", header=None, sep=",\s", names=columns, na_values=["?"], engine="python")


In [76]:
dataset

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [77]:
# transform categorical data to numerical data
def transform_data(dataset):
    dataset_transformed = pd.get_dummies(dataset, columns=["sex", "workclass", "education", "marital-status", "occupation", 
                                                           "relationship", "race", "native-country", "income"], drop_first=False)
    dataset_transformed = dataset_transformed.drop(columns=["sex_Female", "income_<=50K", "workclass_Private", "education_HS-grad", 
                                                            "marital-status_Married-civ-spouse", "occupation_Prof-specialty" , "relationship_Husband", 
                                                            "race_White", "native-country_United-States"])
    dataset_transformed = dataset_transformed.drop(columns=["education-num"])
    return dataset_transformed

dataset_transformed = transform_data(dataset)

In [78]:
dataset_transformed

,age,fnlwgt,capital-gain,capital-loss,hours-per-week,sex_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Self-emp-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_Vietnam,native-country_Yugoslavia,income_>50K
0,39,77516,2174,0,40,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,50,83311,0,0,13,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,38,215646,0,0,40,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,53,234721,0,0,40,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,28,338409,0,0,40,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,0,0,38,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
32557,40,154374,0,0,40,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
32558,58,151910,0,0,40,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
32559,22,201490,0,0,20,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [79]:
# make logistic regression model to predict income based on all features except sex
model = LogisticRegression()
not_shown_columns = ["sex_Male", "income_>50K", "fnlwgt"]
model.fit(dataset_transformed.drop(columns=not_shown_columns, axis=1), dataset_transformed["income_>50K"], sample_weight=dataset_transformed["fnlwgt"])


/home/kamil/szkola/xai/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [80]:
# loading test set
dataset_test = pd.read_csv("datasets/datasets/adult/raw/adult.test", header=None, sep=",\s", names=columns, skiprows=1, na_values="?")
dataset_test["income"] = dataset_test["income"].str.replace(".", "")
dataset_test_transformed = transform_data(dataset_test)
dataset_test_transformed["native-country_Holand-Netherlands"] = 0
dataset_test_transformed = dataset_test_transformed[dataset_transformed.columns]

# scoring the model

print("Model accuracy:", model.score(dataset_test_transformed.drop(columns=not_shown_columns, axis=1), 
                                     dataset_test_transformed["income_>50K"]))
predictions = model.predict(dataset_test_transformed.drop(columns=not_shown_columns, axis=1))


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_8732/298871571.py:2: SyntaxWarning: invalid escape sequence '\s'
  dataset_test = pd.read_csv("datasets/datasets/adult/raw/adult.test", header=None, sep=",\s", names=columns, skiprows=1, na_values="?")
/tmp/ipykernel_8732/298871571.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset_test = pd.read_csv("datasets/datasets/adult/raw/adult.test", header=None, sep=",\s", names=columns, skiprows=1, na_values="?")


Model accuracy: 0.8332412014004054


In [81]:
test_with_predictions = dataset_test.copy()
test_with_predictions["income"] = test_with_predictions["income"] == ">50K"
test_with_predictions["predicted_income"] = predictions
def split_male_female(data):
    return data[data["sex"]=="Male"] , data[data["sex"]=="Female"]
data_male, data_female = split_male_female(test_with_predictions)

In [82]:
def make_crosstab(data):
    return pd.crosstab(data["predicted_income"], data["income"], rownames=["Predicted"], colnames=["Actual"])
crosstab_male = make_crosstab(data_male)

In [83]:
crosstab_female = make_crosstab(data_female)
crosstab_female.to_markdown()

'| Predicted   |   False |   True |\n|:------------|--------:|-------:|\n| False       |    4721 |    326 |\n| True        |     110 |    264 |'

In [84]:
def tp(crosstab):
    return crosstab[True][True]
def fp(crosstab):
    return crosstab[False][True]
def tn(crosstab):
    return crosstab[False][False]
def fn(crosstab):
    return crosstab[True][False]

def get_ratio_of_positives(crosstab):
    return (tp(crosstab) + fp(crosstab)) / (tp(crosstab) + fp(crosstab) + tn(crosstab) + fn(crosstab))

def get_true_positive_rate(crosstab):
    return tp(crosstab) / (tp(crosstab) + fn(crosstab))

def get_positive_predictive_value(crosstab):
    return tp(crosstab) / (tp(crosstab) + fp(crosstab))

def get_false_positive_rate(crosstab):
    return fp(crosstab) / (fp(crosstab) + tn(crosstab))

def get_negative_predictive_value(crosstab):
    return tn(crosstab) / (tn(crosstab) + fn(crosstab))

def analyze_fairness(crosstab_privileged, crosstab_unprivileged):
    print("- Privileged group cross-tab:\n" , crosstab_privileged.to_markdown())
    print("- Unprivileged group cross-tab:\n" , crosstab_unprivileged.to_markdown())
    print("- Demographic parity ratio: ", get_ratio_of_positives(crosstab_privileged) / get_ratio_of_positives(crosstab_unprivileged))
    print("- Equalized odds")
    print("\t- Equal opportunity ratio: ", get_true_positive_rate(crosstab_privileged) / get_true_positive_rate(crosstab_unprivileged))
    print("\t- Predictive equality ratio: ", get_false_positive_rate(crosstab_privileged) / get_false_positive_rate(crosstab_unprivileged))
    print("- Predictive parity")
    print("\t- Positive predictive parity ratio: ", get_positive_predictive_value(crosstab_privileged) / get_positive_predictive_value(crosstab_unprivileged))
    print("\t- Negative predictive parity ratio: ", get_negative_predictive_value(crosstab_privileged) / get_negative_predictive_value(crosstab_unprivileged))

analyze_fairness(crosstab_male, crosstab_female)

- Privileged group cross-tab:
 | Predicted   |   False |   True |
|:------------|--------:|-------:|
| False       |    6817 |   1492 |
| True        |     787 |   1764 |
- Unprivileged group cross-tab:
 | Predicted   |   False |   True |
|:------------|--------:|-------:|
| False       |    4721 |    326 |
| True        |     110 |    264 |
- Demographic parity ratio:  3.404775164711791
- Equalized odds
	- Equal opportunity ratio:  1.2107717221353584
	- Predictive equality ratio:  4.545450958825499
- Predictive parity
	- Positive predictive parity ratio:  0.9796158369266953
	- Negative predictive parity ratio:  0.87708935340081


In [85]:
# make and evaluate model doing the same prediction but with decision tree
model_tree = DecisionTreeClassifier()
model_tree.fit(dataset_transformed.drop(columns=not_shown_columns, axis=1), dataset_transformed["income_>50K"])
print("Model accuracy:", model_tree.score(dataset_test_transformed.drop(columns=not_shown_columns, axis=1), 
                                          dataset_test_transformed["income_>50K"]))

Model accuracy: 0.8189300411522634


In [87]:
test_with_predictions["predicted_income"] = model_tree.predict(dataset_test_transformed.drop(columns=not_shown_columns, axis=1))
data_male, data_female = split_male_female(test_with_predictions)

In [88]:
crosstab_male = make_crosstab(data_male)
crosstab_female = make_crosstab(data_female)
analyze_fairness(crosstab_male, crosstab_female)

- Privileged group cross-tab:
 | Predicted   |   False |   True |
|:------------|--------:|-------:|
| False       |    6513 |   1300 |
| True        |    1091 |   1956 |
- Unprivileged group cross-tab:
 | Predicted   |   False |   True |
|:------------|--------:|-------:|
| False       |    4521 |    247 |
| True        |     310 |    343 |
- Demographic parity ratio:  2.3292111207939556
- Equalized odds
	- Equal opportunity ratio:  1.033337870072564
	- Predictive equality ratio:  2.2359288829308857
- Predictive parity
	- Positive predictive parity ratio:  1.2221245195532384
	- Negative predictive parity ratio:  0.8791540752141698


In [89]:
from dalex.fairness import reweight


In [90]:
weights = reweight(dataset_transformed["sex_Male"], dataset_transformed["income_>50K"])

Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
protected Series will be converted to np.ndarray
protected array is not string type, converting to string 


In [91]:
model.fit(dataset_transformed.drop(columns=not_shown_columns, axis=1), dataset_transformed["income_>50K"], sample_weight=weights)
print("Model accuracy:", model.score(dataset_test_transformed.drop(columns=not_shown_columns, axis=1),
                                        dataset_test_transformed["income_>50K"]))
test_with_predictions["predicted_income"] = model.predict(dataset_test_transformed.drop(columns=not_shown_columns))
data_male, data_female = split_male_female(test_with_predictions)


Model accuracy: 0.8383391683557521


/home/kamil/szkola/xai/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [92]:
analyze_fairness(make_crosstab(data_male), make_crosstab(data_female))

- Privileged group cross-tab:
 | Predicted   |   False |   True |
|:------------|--------:|-------:|
| False       |    7055 |   1556 |
| True        |     549 |   1700 |
- Unprivileged group cross-tab:
 | Predicted   |   False |   True |
|:------------|--------:|-------:|
| False       |    4495 |    191 |
| True        |     336 |    399 |
- Demographic parity ratio:  1.5273961739391888
- Equalized odds
	- Equal opportunity ratio:  0.7720468246784037
	- Predictive equality ratio:  1.0380732415270157
- Predictive parity
	- Positive predictive parity ratio:  1.3924317240410944
	- Negative predictive parity ratio:  0.8541143471067932
